In [1]:
## module needed

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import librosa
import os

import pickle

In [41]:
## function for splitting the song into equal size (1000)

def cut_song(song):
    start = 0
    end = len(song)

    song_pieces = []

    while start + 100000 < end:
        song_pieces.append(song[start:start+100000])

    start += 100000

    return song_pieces

In [42]:
## function for converting into melspectrograms

def prepare_song(song_path):
    list_matrices = []
    y,sr = librosa.load(song_path,sr=22050)
    song_pieces = cut_song(y)
    for song_piece in song_pieces:
        melspect = librosa.feature.melspectrogram(song_piece)
        list_matrices.append(melspect)
    return list_matrices

In [10]:
## getting all the genre

genre = os.listdir('tracks')
print(genre)

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [26]:
X = []
y = []

for i in range(len(genre)):
    for song_name in os.listdir(os.path.join('tracks',genre[i])):
        try:
            song_pieces = prepare_song(os.path.join('tracks', genre[i], song_name))
            X += song_pieces
            y += ([i]*len(song_pieces))
        except Exception:
            print(f"unable to add this file {os.path.join('tracks', genre[i], song_name)}")
        
    print(f"{i+1}/{len(genre)} done")

1/10 done
2/10 done
3/10 done
4/10 done
5/10 done


/home/sunil/.local/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


unable to add this file tracks/jazz/jazz.00054.wav
6/10 done
7/10 done
8/10 done
9/10 done
10/10 done


In [30]:
## preserving the X and y

with open('X.pickle', 'wb') as f:
    pickle.dump(X, f)

with open('y.pickle', 'wb') as f:
    pickle.dump(y, f)

In [2]:
## loading the data back

X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

## train / testing

In [3]:
## packages

import numpy as np
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

2022-05-16 21:32:42.807040: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X).reshape(-1, 128, 196, 1), 
                                                    np.array(y),
                                                    test_size=0.33,
                                                    random_state=42)

In [6]:
## model design

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 196, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
# since ther are 10 labels

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 194, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 97, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 95, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 47, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 45, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 80640)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                5

2022-05-16 21:33:59.541100: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-16 21:33:59.568073: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-16 21:34:01.503434: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-16 21:34:01.514777: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2022-05-16 21:34:01.515368: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-16 21:34:01.568509: I tensorflow/stream_executor/platfo

In [10]:

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics='accuracy')

history = model.fit(X_train, y_train, epochs=10)

2022-05-16 21:41:22.455554: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 402913280 exceeds 10% of free system memory.


Epoch 1/10


2022-05-16 21:41:31.294754: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 100122624 exceeds 10% of free system memory.
2022-05-16 21:41:40.798440: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 100122624 exceeds 10% of free system memory.


  1/126 [..............................] - ETA: 25:20 - loss: 22.4154 - accuracy: 0.1562

2022-05-16 21:41:41.310484: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 100122624 exceeds 10% of free system memory.


  4/126 [..............................] - ETA: 5:20 - loss: -1.3045 - accuracy: 0.1224

: 

: 